In [12]:
# Memanggil library yang dibutuhkan
import ee
import geemap

# Autentikasi dan Inisialisasi
ee.Authenticate()
ee.Initialize()

# Membuat peta interaktif
Map = geemap.Map()

# --- GANTI DENGAN PATH ASET BARU ANDA UNTUK TAHUN 2020 ---
path_aset_lc_2020 = "users/ivan_taslim/LC_Paguyaman_2020_DW_Generated"

# Memuat raster tutupan lahan 2020 yang baru
lc_2020 = ee.Image(path_aset_lc_2020)
aoi = lc_2020.geometry()

# Menampilkan AOI di peta
Map.addLayer(aoi, {'color': 'red'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)
Map

Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [13]:
# Memuat data properti tanah dari SoilGrids dan memilih lapisan teratas
sand = ee.Image("projects/soilgrids-isric/sand_mean").select(0).clip(aoi)
silt = ee.Image("projects/soilgrids-isric/silt_mean").select(0).clip(aoi)
clay = ee.Image("projects/soilgrids-isric/clay_mean").select(0).clip(aoi)

# Mengestimasi Grup Hidrologi Tanah (HSG)
hsg = ee.Image(4).where(sand.gt(50), 1) \
                 .where(silt.gt(50), 2) \
                 .where(clay.gt(40), 3) \
                 .rename('hsg')

# Menampilkan peta HSG
hsg_palette = ['#FFFF00', '#00FF00', '#0000FF', '#FF0000']
Map.addLayer(hsg, {'min': 1, 'max': 4, 'palette': hsg_palette}, 'Peta Grup Hidrologi Tanah (HSG)')

print("Peta Grup Hidrologi Tanah (HSG) berhasil dibuat.")

Peta Grup Hidrologi Tanah (HSG) berhasil dibuat.


In [15]:
# Kamus penerjemah Curve Number (CN)
lc_values = [1, 2, 3, 4, 6] # Sesuai skema reklasifikasi baru
cn_values_by_hsg = [
    [30, 55, 70, 77], # 1: Hutan
    [49, 69, 79, 84], # 2: Lahan terbuka
    [72, 81, 88, 91], # 3: Sawah
    [67, 78, 85, 89], # 4: Pertanian
    [89, 92, 94, 95]  # 6: Area terbangun & Air
]

# Proses pembuatan Peta CN
cn_map_2020 = ee.Image(0).rename('cn')
for i in range(len(lc_values)):
    lc_class = lc_values[i]
    cn_hsg_a, cn_hsg_b, cn_hsg_c, cn_hsg_d = cn_values_by_hsg[i]
    
    cn_map_2020 = cn_map_2020.where(lc_2020.eq(lc_class).And(hsg.eq(1)), cn_hsg_a) \
                             .where(lc_2020.eq(lc_class).And(hsg.eq(2)), cn_hsg_b) \
                             .where(lc_2020.eq(lc_class).And(hsg.eq(3)), cn_hsg_c) \
                             .where(lc_2020.eq(lc_class).And(hsg.eq(4)), cn_hsg_d)

# Menambahkan masker untuk menghilangkan piksel nol
cn_map_2020 = cn_map_2020.updateMask(cn_map_2020.gt(0))

# Menampilkan peta CN
cn_palette = ['#32CD32', '#FFFF00', '#FFA500', '#FF0000']
Map.addLayer(cn_map_2020, {'min': 30, 'max': 100, 'palette': cn_palette}, 'Peta Curve Number 2020')

print("Peta Curve Number (CN) untuk tahun 2020 berhasil dibuat.")

Peta Curve Number (CN) untuk tahun 2020 berhasil dibuat.


In [16]:
# Definisikan skenario curah hujan dalam 24 jam (dalam mm)
P = 100

# Menghitung Potensi Retensi Maksimum (S)
S = (ee.Image(1000).divide(cn_map_2020).subtract(10)).multiply(25.4)
# Menghitung Abstraksi Awal (Ia)
Ia = S.multiply(0.2)

# Menghitung Limpasan Permukaan (Q) dengan metode aman
P_img = ee.Image(P)
numerator = (P_img.subtract(Ia)).pow(2)
denominator = (P_img.subtract(Ia)).add(S)
potential_Q = numerator.divide(denominator)
runoff_condition = P_img.gt(Ia)
Q_2016 = ee.Image(0).where(runoff_condition, potential_Q).rename('runoff_mm')

# Menghitung total volume limpasan dalam meter kubik (m3)
pixel_area_m2 = ee.Image.pixelArea()
runoff_m3 = Q_2020.divide(1000).multiply(pixel_area_m2)
total_runoff_volume_2020 = runoff_m3.reduceRegion(
    reducer=ee.Reducer.sum(), geometry=aoi, scale=30, maxPixels=1e12
)

# Menampilkan hasil
print("======================================================================")
print(f"HASIL ANALISIS ULANG LIMPASAN PERMUKAAN TAHUN 2020")
print(f"(Skenario Hujan: {P} mm/24 jam)")
print("----------------------------------------------------------------------")
print(f"Total Volume Limpasan: {total_runoff_volume_2020.get('runoff_mm').getInfo():,.2f} meter kubik")
print("======================================================================")

# Menampilkan peta limpasan
runoff_palette = ['#FFFFFF', '#FFFF00', '#FFA500', '#FF0000', '#800000']
Map.addLayer(Q_2020, {'min': 0, 'max': 80, 'palette': runoff_palette}, 'Peta Limpasan Permukaan 2020 (mm)')

HASIL ANALISIS ULANG LIMPASAN PERMUKAAN TAHUN 2020
(Skenario Hujan: 100 mm/24 jam)
----------------------------------------------------------------------
Total Volume Limpasan: 94,824,895.37 meter kubik


In [17]:
# ======================================================================
# LANGKAH 5: EKSPOR PETA LIMPASAN 2020
# ======================================================================

# 5.1 Ekspor ke GEE Assets
task_asset = ee.batch.Export.image.toAsset(
  image=Q_2020.toFloat(), # Konversi ke float untuk ekspor
  description='Limpasan_Paguyaman_2020_to_Asset',
  assetId='users/ivan_taslim/Limpasan_Paguyaman_2020', # Path aset tujuan Anda
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_asset.start()
print("Proses ekspor ke GEE ASSETS telah dimulai.")

# 5.2 Ekspor ke Google Drive
task_drive = ee.batch.Export.image.toDrive(
  image=Q_2020.toFloat(),
  description='Limpasan_Paguyaman_2020_to_Drive',
  folder='Hasil_Analisis_DAS_Paguyaman',
  fileNamePrefix='Limpasan_Paguyaman_2020',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_drive.start()
print("Proses ekspor ke GOOGLE DRIVE telah dimulai.")
print("\nPERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.")

Proses ekspor ke GEE ASSETS telah dimulai.
Proses ekspor ke GOOGLE DRIVE telah dimulai.

PERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.
